In [ ]:
# importing the libraries
import numpy as np

# for reading and displaying images
import matplotlib.pyplot as plt
%matplotlib inline

# for creating validation set
from sklearn.model_selection import train_test_split

# for evaluating the model
from sklearn.metrics import accuracy_score

# PyTorch libraries and modules
import torch
from torch.nn import Linear, ReLU, Sequential, Conv2d, MaxPool2d, Module, BatchNorm2d, CrossEntropyLoss, Softmax
from torch.optim import Adam

In [ ]:
import torchvision
import torchvision.transforms as transforms
transform = transforms.ToTensor()

train_set = torchvision.datasets.MNIST(
    root='./data/', train=True, transform=transform, download=True)
val_set = torchvision.datasets.MNIST(
    root='./data/', train=False, transform=transform, download=True)
train_set.classes

In [ ]:
def show_images(images):
    fig = plt.figure(figsize=(64, 64))

    for i in range(len(images)):
        sub = fig.add_subplot(32, 1, i + 1)
        sub.imshow(images[i])

show_images(train_set.data[:5])
train_set.targets[:5]

In [ ]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=32)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=len(val_set.data))

In [ ]:
# Define CNN model: 2 convolutional layers
class ConvNet(Module):   
    def __init__(self):
        super().__init__()

        self.cnn_layers = Sequential(
            # shape is (batch_size, 1, 28, 28)
            Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1),
            # shape is (batch_size, 16, 28, 28)
            BatchNorm2d(num_features=16),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            # shape is (batch_size, 16, 14, 14)
            Conv2d(in_channels=16, out_channels=16, kernel_size=3, stride=1, padding=1),
            # shape is (batch_size, 16, 14, 14)
            BatchNorm2d(num_features=16),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            # shape is (batch_size, 16, 7, 7)
        )

        self.fc = Linear(in_features=16 * 7 * 7, out_features=10)

    # Defining the forward pass    
    def forward(self, x):
        # shape is (batch_size, 1, 28, 28)
        x = self.cnn_layers(x)
        # shape is (batch_size, 16, 7, 7)
        x = x.view(x.size(0), -1)
        # shape is (batch_size, 16 * 7 * 7)
        x = self.fc(x)
        # shape is (batch_size, 10)
        return x

In [ ]:
model = ConvNet()
criterion = torch.nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)
train_losses = []
val_losses = []
running_train_loss = 0.0

val_images, val_targets = next(iter(val_loader))
if torch.cuda.is_available():
    val_images = val_images.cuda()
    val_targets = val_targets.cuda()

for epoch in range(3):
    for batch_id, (images, targets) in enumerate(train_loader):
        optimizer.zero_grad()

        if torch.cuda.is_available():
            images = images.cuda()
            targets = targets.cuda()
            
        outputs = model(images)
        loss = criterion(outputs, targets)
        loss.backward()
        running_train_loss += loss.item()
        optimizer.step()
        
        if batch_id and batch_id % 200 == 0:
            val_outputs = model(val_images)
            val_loss = criterion(val_outputs, val_targets)
            val_losses.append(val_loss)
            
            running_train_loss /= 200
            train_losses.append(running_train_loss)
            print(f'Epoch {epoch} Batch {batch_id}: Loss is {running_train_loss} Validation loss is: {val_loss}')
            running_train_loss = 0

print('Finished Training')

In [ ]:
# plot the training and validation losses
plt.plot(train_losses, label='Training loss')
plt.plot(val_losses, label='Validation loss')
plt.legend()
plt.show()

In [ ]:
def accuracy_on_a_set(images, labels):
    with torch.no_grad():
        output = model(images)
    
    softmax = Softmax(dim=1)(output).cpu()
    predictions = np.argmax(softmax, axis=1)

    return accuracy_score(labels, predictions)

full_train_images, full_train_targets = next(iter(torch.utils.data.DataLoader(train_set, batch_size=len(train_set.data))))
print('accuracy on training set', accuracy_on_a_set(full_train_images, full_train_targets))
print('accuracy on validation set', accuracy_on_a_set(val_images, val_targets))